In [1]:
%%punch_spark_session
{
    "spark.hadoop.fs.s3a.endpoint": "http://minio.object-store:9000",
    "spark.hadoop.fs.s3a.access.key": "minioadmin",
    "spark.hadoop.fs.s3a.secret.key": "minioadmin",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
}

22/10/19 17:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:SparkMonitorKernel:Client Connected ('127.0.0.1', 54162)


### Generate data to save

In [2]:
%%punch_source --type generator --name df -o
messages:
        - data: My first message
        - data: My second message
        - data: My third one

Data is available in df variable.


In [3]:
df.head(2)

[Row(data='My first message'), Row(data='My second message')]

### ElasticSearch

##### Save to elasticsearch

In [4]:
%%punch_sink --type elasticsearch -df df
http_hosts:
  - host: elasticsearch-master.doc-store
    port: 9200
    scheme: http
index:
    type: constant
    value: my-index-pyspark
security: # Default null 
  # Credentials, token or basic
  credentials: # Default null
    # Option 1 : Basic
    username: "elastic"
    password: "elastic"

Data saved.


#####  Fetch data from elasticsearch

In [5]:
%%punch_source --type elasticsearch --name out -o
http_hosts:
  - host: elasticsearch-master.doc-store
    port: 9200
    scheme: http 
security: # Default null 
  # Credentials, token or basic
  credentials: # Default null
    # Option 1 : Basic
    username: "elastic"
    password: "elastic"

# Source settings
index: my-index-pyspark

Data is available in out variable.


In [6]:
out.head(2)

[Row(data='My second message'), Row(data='My third one')]

In [7]:
del out

### Kafka

##### Save to kafka

In [8]:
%%punch_sink --type kafka -df df
kafka.bootstrap.servers: kooker-kafka-kafka-bootstrap.processing:9092
topic: my-topic-pyspark

Data saved.


### Fetch data from kafka

In [9]:
%%punch_source --type kafka --name out -o
kafka.bootstrap.servers: kooker-kafka-kafka-bootstrap.processing:9092
topic: my-topic-pyspark

Data is available in out variable.


In [10]:
out.head()

Row(key=None, value=bytearray(b'{"data":"My second message"}'), topic='my-topic-pyspark', partition=0, offset=63, timestamp=datetime.datetime(2022, 10, 18, 14, 12, 2, 559000), timestampType=0)

In [11]:
del out

### File (s3)

##### Save to file in s3

In [12]:
%%punch_sink --type file -df df
options:
    header: True
format: csv
path: s3a://default/subdirectory/test.csv
save_mode: Overwrite

22/10/19 17:59:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/10/19 17:59:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


Data saved.


##### Recover data from s3 (file node)

In [13]:
%%punch_source --type file --name s3 -o 
options:
    header: True
path: s3a://default/subdirectory/test.csv

Data is available in s3 variable.


In [14]:
s3.head(2)

[Row(_c0='0', data='My first message'), Row(_c0='1', data='My second message')]